In [204]:
import pandas as pd
import calmap
import numpy as np
import os

In [205]:
from pathlib import Path
from os import environ, write

import re
from datetime import datetime, time
import datetime as dt
from collections import defaultdict

import requests

In [323]:
def load_data():
    if environ.get("USE_LOCAL_FILES"):
        paths = Path("extracted").rglob("*.csv")
        df_parts = []

        for path in sorted(paths):

            print("Reading for analysis:", path)

            with open(path, "r", encoding="utf-8") as fp:
                df_part = pd.read_csv(fp, delimiter=",", dtype=str)
                df_parts.append(df_part)

            # if df is None:
            #     df = df_part
            # else:
            #     df = df.append(df_part, ignore_index=True)

        df = pd.concat(df_parts, ignore_index=True)

    else:
        raise NotImplementedError("Remote data not yet implemented")
    
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors = 'coerce')
    df['date_of_proceeding'] = pd.to_datetime(df['date_of_proceeding'], errors = 'coerce')
    df['date_of_publication'] = pd.to_datetime(df['date_of_publication'], errors = 'coerce')

    return df

In [324]:
df = load_data()

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-17T08-05-41.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-24T08-05-43.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-31T03-50-40.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-07T03-47-56.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-14T03-54-29.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-21T04-08-08.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-28T04-20-16.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-03-07T07-20-48.jsonl.csv
Reading for analysis: extracted/buckets/insolven

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T00-26-46.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T07-31-07.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T13-43-00.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T20-17-09.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T06-38-42.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T13-46-12.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T19-37-11.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-22T07-02-46.jsonl.csv
Reading for analysis: extracted/buckets/insolven

In [325]:
df.drop_duplicates(subset ="description_hash", keep = "last", inplace = True)

In [326]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 852091 entries, 159 to 1844472
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 34818 non-null   object        
 1   case_nr              852091 non-null  object        
 2   court                852091 non-null  object        
 3   date_of_birth        610830 non-null  datetime64[ns]
 4   date_of_proceeding   754557 non-null  datetime64[ns]
 5   date_of_publication  852091 non-null  datetime64[ns]
 6   description_hash     852091 non-null  object        
 7   federal_state        852091 non-null  object        
 8   file_name            430671 non-null  object        
 9   kind                 851544 non-null  object        
 10  title                430671 non-null  object        
 11  type_of_proceeding   852091 non-null  object        
 12  zipcode              847993 non-null  object        
 13  detail_form

In [327]:
for plz in df.zipcode.dropna().unique():
    if isinstance(plz, float): 
        print(plz, type(plz))

In [328]:
unique_plzs = df.zipcode.dropna().unique()
unique_plzs.sort()
unique_plzs

array(['00000', '00037', '00165', ..., '99996', '99998', '99999'],
      dtype=object)

# Bundesweit

### Filtern nach Insolvenzart

In [329]:
df1 = df.replace(to_replace = ["nw"],
                   value ="Nordrhein-Westfalen")

In [330]:
df2 = df1.replace(to_replace = ["bb"],
                   value ="Brandenburg")

In [331]:
df3 = df2.replace(to_replace = ["sh"],
                   value ="Schleswig-Holstein")

In [332]:
df4 = df3.replace(to_replace = ["ns"],
                   value ="Niedersachsen")

In [333]:
df5 = df4.replace(to_replace = ["by"],
                   value ="Bayern")

In [334]:
df6 = df5.replace(to_replace = ["rp"],
                   value ="Rheinland-Pfalz")

In [335]:
df7 = df6.replace(to_replace = ["sn"],
                   value ="Sachsen")

In [336]:
df8 = df7.replace(to_replace = ["bw"],
                   value ="Baden-Württemberg")

In [337]:
df9 = df8.replace(to_replace = ["be"],
                   value ="Berlin")

In [338]:
df10 = df9.replace(to_replace = ["st"],
                   value ="Brandenburg")

In [339]:
df11 = df10.replace(to_replace = ["he"],
                   value ="Hessen")

In [340]:
df12 = df11.replace(to_replace = ["hh"],
                   value ="Hamburg")

In [341]:
df13 = df12.replace(to_replace = ["sl"],
                   value ="Saarland")

In [342]:
df14 = df13.replace(to_replace = ["hb"],
                   value ="Bremen")

In [343]:
df15 = df14.replace(to_replace = ["th"],
                   value ="Thüringen")

In [344]:
df16 = df15.replace(to_replace = ["mv"],
                   value ="Mecklenburg-Vorpommern")

In [345]:
# IN = Regelinsolvenz

def in_kind(df16):
    in_unternehmen = (df16[df16['kind'] == 'in'])
    return in_unternehmen

# Funktion def wird ausgeführt
data_in = in_kind(df16)

In [346]:
def in_eröffnung(data_in):
    in_eröffnet = data_in[data_in['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen'])]
    return in_eröffnet

# Funktion def wird ausgeführt
data_eröffnet_in = in_eröffnung(data_in)

In [244]:
data_nw_ik = data_eröffnet_in[data_eröffnet_in["federal_state"].isin(['Nordrhein-Westfalen'])]

In [347]:
date_range_nw_ik = data_eröffnet_in[data_eröffnet_in.date_of_publication.between('2021-08-01', '2021-11-07')]

In [348]:
date_range_nw_ik.head()

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,description_hash,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
987532,NaN,IN 87/21,Mühlhausen,1985-10-11,2021-08-05,2021-08-05,519f850089d8a6dcd10dacfe74ef84e8ecabb68edaf44a...,Thüringen,NaN,in,NaN,Eröffnungen,99734,tbl_ergebnis:9:frm_detail,neu,"Lingner, Mike",NaN,1aa89f8929792316eb3eda78eef3ccc21d6a8c7837a9ff...
987647,NaN,56 IN 69/21,Flensburg,1990-01-20,2021-08-04,2021-08-05,f6f43a6bc328f47fce7291bbedcf9eed64a022a24a7b18...,Schleswig-Holstein,NaN,in,NaN,Eröffnungen,24937,tbl_ergebnis:12:frm_detail,neu,"Youssef, Adam",NaN,1deba6b8fe6197739645467f4fa035e5c754b42cd0877c...
987648,NaN,1 IN 43/21,Schwarzenbek,1964-05-26,2021-08-04,2021-08-05,7f940a632c814507b2e3e632628fd9dad52c7c12a0252f...,Schleswig-Holstein,NaN,in,NaN,Eröffnungen,23879,tbl_ergebnis:11:frm_detail,neu,Warncke,NaN,96c5e0f5c67edd9679f9cfbac9497fb5154ffe0d3a49d9...
987649,NaN,66 IN 40/21,Norderstedt,1986-12-07,2021-07-23,2021-08-05,ab72bfd1695ed8452284bb78638e52b323df4b5aaa0d61...,Schleswig-Holstein,NaN,in,NaN,Eröffnungen,22846,tbl_ergebnis:10:frm_detail,neu,Tirelioglu,NaN,9db886d26348c2a8ebd2c6110c08e7078479356c46799b...
987650,NaN,91 IN 49/21,Neumünster,1985-09-11,2021-08-05,2021-08-05,4cd0e939b02d08dd8a2009349404cf6d92eaba34b56dd9...,Schleswig-Holstein,NaN,in,NaN,Eröffnungen,24537,tbl_ergebnis:9:frm_detail,neu,"Thormann-Neubauer, Tim",NaN,82761562a61c868e25ffc926fa45b2b82b06438345d3b4...


In [349]:
eröffnung_court_nw = date_range_nw_ik.groupby(['zipcode'], as_index=False).count()[['zipcode','description_hash']]
eröffnung_court_nw.sample(10)

,zipcode,description_hash
644,20148,1
2581,75173,2
1166,32549,1
532,16818,1
47,02957,1
962,27246,2
2471,72280,2
1161,32361,1
751,22844,2
738,22547,3


In [350]:
"""for plz in eröffnung_court_nw.zipcode:
    if not isinstance(plz, str):
        print(plz, type(plz))"""

'for plz in eröffnung_court_nw.zipcode:\n    if not isinstance(plz, str):\n        print(plz, type(plz))'

In [353]:
df_osm_plz = pd.read_csv("plz_de.csv", dtype=str)
df_osm_plz = df_osm_plz[["plz", "ags", "einwohner", "landkreis", "ort"]]

df_osm_plz.ags = df_osm_plz.ags.str.slice(0, 5)

df_osm_plz.rename(columns={"plz": "zipcode"}, inplace=True)
df_osm_plz = df_osm_plz.groupby("zipcode").last()


joined = eröffnung_court_nw.join(df_osm_plz, on=["zipcode"], how="left")
joined.sample(50)

,zipcode,description_hash,ags,einwohner,landkreis,ort
2703,79117,2,08311,18272,None,Freiburg im Breisgau
2580,75172,4,08231,13892,None,Pforzheim
2403,70374,1,08111,24823,None,Stuttgart
1865,52531,2,05370,24238,Kreis Heinsberg,Übach-Palenberg
905,25927,1,01054,1722,Kreis Nordfriesland,Aventoft
1536,44575,2,05562,22186,Kreis Recklinghausen,Castrop-Rauxel
3140,93128,1,09375,15061,Landkreis Regensburg,Regenstauf
249,10535,1,NaN,NaN,NaN,NaN
672,21217,1,03353,16607,Landkreis Harburg,Seevetal
2510,73257,1,08116,9424,Landkreis Esslingen,Köngen


In [354]:
joined.einwohner = joined.einwohner.astype(float)

In [251]:
joined["ratio"] = joined["description_hash"]/joined["einwohner"]*1000

In [196]:
#joined.to_csv('eröffnung_zip_nw_90T_IK_1000.csv')

In [210]:
joined = joined.drop(columns=["zipcode", "ratio"])

KeyError: "['ratio'] not found in axis"

In [198]:
#joined = joined.dropna

In [355]:
joined = joined.groupby(["ags", "landkreis"], as_index= False).sum()

In [356]:
joined.head()

,ags,landkreis,description_hash,einwohner
0,01051,Kreis Dithmarschen,19,79635.0
1,01053,Kreis Herzogtum Lauenburg,12,126782.0
2,01054,Kreis Nordfriesland,19,63147.0
3,01055,Kreis Ostholstein,12,112506.0
4,01056,Kreis Pinneberg,17,184139.0


In [357]:
joined[joined['ags'].isna()]

,ags,landkreis,description_hash,einwohner


In [358]:
joined.isnull().sum() # 21,57 % der Werte fehlen !

ags                 0
landkreis           0
description_hash    0
einwohner           0
dtype: int64

In [321]:
joined["ratio"] = joined["description_hash"]/joined["einwohner"]*100000

In [322]:
joined.to_csv('90T_joined_in_eröffnet_bund_kreis_100000.csv')

In [312]:
len(joined.landkreis.unique())

292

In [30]:
nw_eröffnet_ik_week = (
    data_nw_ik[["date_of_publication", "description_hash"]]
    .groupby([pd.Grouper(key="date_of_publication", freq="W-MON", label="left")])
    .count()
)

In [44]:
df.federal_state.unique()

array(['bb', 'nw', 'sh', 'ns', 'by', 'sn', 'bw', 'be', 'st', 'he', 'rp',
       'hb', 'mv', 'hh', 'sl', 'th', 'Bayern', 'Rheinland-Pfalz',
       'Thüringen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Sachsen',
       'Saarland', 'Nordrhein-Westfalen', 'Niedersachsen',
       'Mecklenburg-Vorpommern', 'Hessen', 'Hamburg', 'Bremen',
       'Brandenburg', 'Berlin', 'Baden-Württemberg'], dtype=object)

In [ ]:
# IK = Verbraucherinsolvenz

def ik_kind(df):
    ik_verbraucher = (df[df['kind'] == 'ik'])
    return ik_verbraucher

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

### Art des Verfahrens

In [18]:
df.type_of_proceeding.unique()

array(['Entscheidungen_im_Verfahren', 'Eroeffnung',
       'Entscheidungen_im_Restschuldbefreiungsverfahren', 'Termine',
       'Entscheidungen_nach_Aufhebung',
       'Verwalter_Treuhaender_Verteilungsverzeichnis',
       'Entscheidungen_im_Verfahren_Aufhebung_Einstellung',
       'Restschuldbefreiung', 'Entscheidungen_im_Verfahren_mit_Termine',
       'Sonstiges', 'Bestimmung_Termine', 'InsO_d_Verw_Treuh_',
       'Sonstiges_ausserhalb_des_Verfahresn', 'Eroeffnungen',
       'Eroeffnung_Insolvenzverfahren',
       'Restschuldbefreiung_Erteilung_Versagung',
       'Abweisungen_mangels_Masse', 'Sicherungsmassnahmen',
       'Anordnung_Sicherungsmassnahmen',
       'Verteilungsverzeichnis_Nachtragsvert_Loeschung',
       'Entscheidungen_nach_Aufhebung_des_Verfahrens',
       'Sicherungsmassnahme_Aufhebung', 'InsO_Plan_Ueberwachung',
       'InsO_Plan_Aufhebung_Ueberwachung', 'Ueberwachte_Insolvenzplaene',
       'InsO_Plan_Ankuendigung_Ueberwachung',
       'Verteilungsverzeichnisse_(§_

In [25]:
def in_eröffnung(data_in):
    in_eröffnet = data_in[data_in['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen'])]
    return in_eröffnet

# Funktion def wird ausgeführt
data_eröffnet_in = in_eröffnung(data_in)

In [36]:
data_eröffnet_in.to_csv('data_eröffnung_in_bund.csv')

# Verbraucher

In [ ]:
def ik_eröffnung(df):
    ik_eröffnet = df[df['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen'])]
    return ik_eröffnet

# Funktion def wird ausgeführt
data_eröffnet = ik_eröffnung(df)

In [ ]:
def ik_masse(df):
    ik_mangels_masse = df[df['type_of_proceeding'].isin(['Abweisungen_mangels_Masse'])]
    return ik_mangels_masse

# Funktion def wird ausgeführt
data_masse = ik_masse(df)

In [ ]:
def ik_sicher(df):
    ik_sicherung = df[df['type_of_proceeding'].isin(['Anordnung_Sicherungsmassnahmen', 'Sicherungsmassnahmen'])]
    return ik_sicherung

# Funktion def wird ausgeführt
data_sicher = ik_sicher(df)

### groupby Bundesland descrptionhash

### groupby zipcode Bundesweit

In [ ]:
freq_in_nw_eröffnung_zipcode = data_nw_eröffnet.groupby(['zipcode'], as_index=False).count()
freq_in_nw_eröffnung_zipcode.sample(10)

### Groupby Inolvenzgericht

### Groupby week

In [32]:
data_eröffnet_in_week = (
    data_eröffnet_in[["date_of_publication", "description_hash"]]
    .groupby(pd.Grouper(key="date_of_publication", freq="W-MON", label="left"))
    .count()
)

In [33]:
data_eröffnet_in_week.head()

,description_hash
date_of_publication,
2018-12-31,392
2019-01-07,249
2019-01-14,231
2019-01-21,233
2019-01-28,445


In [35]:
data_eröffnet_in_week.to_csv('freq_in_eröffnung_bund.csv')

### Alter 

In [ ]:
freq_in_nw_eröffnung_birth = freq_in_nw_eröffnung_birth.groupby(['date_of_birth'], as_index=False).count()
freq_in_nw_eröffnung_birth.sample(10)

In [6]:
data_nw = df[df["federal_state"].isin(['Nordrhein-Westfalen', "nw"])]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204015 entries, 1341 to 1819821
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 8959 non-null    object        
 1   case_nr              204015 non-null  object        
 2   court                204015 non-null  object        
 3   date_of_birth        103479 non-null  datetime64[ns]
 4   date_of_proceeding   203559 non-null  datetime64[ns]
 5   date_of_publication  204015 non-null  datetime64[ns]
 6   description_hash     204015 non-null  object        
 7   federal_state        204015 non-null  object        
 8   file_name            101866 non-null  object        
 9   kind                 203915 non-null  object        
 10  title                101866 non-null  object        
 11  type_of_proceeding   204015 non-null  object        
 12  zipcode              203738 non-null  object        
 13  detail_for

In [7]:
data_nw.isnull().sum() # die Hälfte der Einträge hat keinen Wert in den geburtsdaten. Es macht also keinen Sinn hier eine Berechnung anzuwenden, da die Fehlerquote ienfach hoch wäre.

_key                   195056
case_nr                     0
court                       0
date_of_birth          100536
date_of_proceeding        456
date_of_publication         0
description_hash            0
federal_state               0
file_name              102149
kind                      100
title                  102149
type_of_proceeding          0
zipcode                   277
detail_form_name       101866
format                 101866
name                   101866
register               187410
request_fingerprint    101866
dtype: int64

In [9]:
def nw_eröffnung(data_nw):
    nw_eröffnet = data_nw[data_nw['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren'])]
    return nw_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = nw_eröffnung(data_nw)

In [11]:
data_nw_eröffnet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32800 entries, 21335 to 1818646
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   _key                 507 non-null    object        
 1   case_nr              32800 non-null  object        
 2   court                32800 non-null  object        
 3   date_of_birth        26980 non-null  datetime64[ns]
 4   date_of_proceeding   32666 non-null  datetime64[ns]
 5   date_of_publication  32800 non-null  datetime64[ns]
 6   description_hash     32800 non-null  object        
 7   federal_state        32800 non-null  object        
 8   file_name            9769 non-null   object        
 9   kind                 32781 non-null  object        
 10  title                9769 non-null   object        
 11  type_of_proceeding   32800 non-null  object        
 12  zipcode              32790 non-null  object        
 13  detail_form_name     2303

In [10]:
data_nw_eröffnet.isnull().sum() # 21,57 % der Werte fehlen !

_key                   32293
case_nr                    0
court                      0
date_of_birth           5820
date_of_proceeding       134
date_of_publication        0
description_hash           0
federal_state              0
file_name              23031
kind                      19
title                  23031
type_of_proceeding         0
zipcode                   10
detail_form_name        9769
format                  9769
name                    9769
register               27911
request_fingerprint     9769
dtype: int64